## 🚧 Planning
### Objective
O objetivo é criar uma ferramenta que leia um arquivo de transações (CSV ou Excel), processe esses dados, categorize os gastos automaticamente e gere gráficos interativos.

### Modules
- Pandas
- Matplotlib
- Plotly
- Streamlit

### Steps
- [x] Use Koggle to take a test database
- [x] Pandas to threating datas
- [x] Categorize datas (using keywords)
- [x] Statistic analysis
    - [x] calculate total expenses
    - [x] daily expenses average
    - [x] the category that user much expense
- [x] Visualizate (charts) 
    - [x] 1. total expenses x money gained
        - [x] quantity numbers
    - [x] 2. daily expenses
    - [x] 3. invested categories
- [ ] Datetype validator
- ...
- [ ] Make a csv uploader (streamlit)
- [ ] Make a spends predictor (AI)

In [229]:
import pandas as pd

df = pd.read_csv('../database/expense_data_1.csv')

# df = df.drop(columns=['Subcategory', 'Note.1', 'Currency', 'Account'])
df = df[['Date', 'Category', 'INR', 'Income/Expense']]

display(df)

,Date,Category,INR,Income/Expense
0,3/2/2022 10:11,Food,50.0,Expense
1,3/2/2022 10:11,Other,300.0,Expense
2,3/1/2022 19:50,Food,78.0,Expense
3,3/1/2022 18:56,Transportation,30.0,Expense
4,3/1/2022 18:22,Food,67.0,Expense
...,...,...,...,...
272,11/22/2021 14:16,Food,90.0,Expense
273,11/22/2021 14:16,Food,97.0,Expense
274,11/21/2021 17:07,Transportation,130.0,Expense
275,11/21/2021 15:50,Food,875.0,Expense


In [230]:
# Validating that date is complete

date_range = []
for d in df['Date'].unique():
    quant = len(str(d))+1
    if not quant in date_range:
        date_range.append(quant)

display(date_range)

[15, 16, 14, 17]

In [231]:
# Organizing dataframe date

# for d in df['Date'].unique():
    # last = str(d)[-5:]
    # if '/' in last:
    #     print('TIME_ERROR')

for i, d in enumerate(df['Date']):
    df.at[i, 'Date'] = d[0:-5].strip()
    
    day = ''
    month = ''
    year = ''

    bar_count = 0
    for caractere in df['Date'][i]:
        if caractere != '/':
            if bar_count == 0:
                month += caractere
            elif bar_count == 1:
                day += caractere
            else:
                year += caractere
        else:
            bar_count += 1
    
    if len(day) == 1:
        day = f'0{day}'
    if len(month) == 1:
        month = f'0{month}'
    if len(year) != 4:
        print('YEAR_ERROR')
    
    df.at[i, 'Date'] = f'{day}/{month}/{year}'

display(df)
display(df.info())

,Date,Category,INR,Income/Expense
0,02/03/2022,Food,50.0,Expense
1,02/03/2022,Other,300.0,Expense
2,01/03/2022,Food,78.0,Expense
3,01/03/2022,Transportation,30.0,Expense
4,01/03/2022,Food,67.0,Expense
...,...,...,...,...
272,22/11/2021,Food,90.0,Expense
273,22/11/2021,Food,97.0,Expense
274,21/11/2021,Transportation,130.0,Expense
275,21/11/2021,Food,875.0,Expense


<class 'pandas.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            277 non-null    str    
 1   Category        277 non-null    str    
 2   INR             277 non-null    float64
 3   Income/Expense  277 non-null    str    
dtypes: float64(1), str(3)
memory usage: 8.8 KB


None

| Are a different currency in df, however will be considered that a unique currency was been used.

In [232]:
# Calculate total expenses

# total_expenses = 0
# for i, transaction in df.iterrows():
#     if transaction['Income/Expense'] == 'Expense':
#         total_expenses += transaction['INR']

total_expenses = df.loc[df['Income/Expense'] == 'Expense']['INR'].sum()

print(f'{total_expenses:.2f}')

59024.00


In [233]:
# Calculate average

expenses_count = df.value_counts(df['Income/Expense'])['Expense']

expenses_daily_average = total_expenses / expenses_count

print(f'{expenses_daily_average:.2f}')

255.52


In [234]:
# Daily expenses chart

import plotly.express as px

dfdate = df.drop(columns=['Category', 'Income/Expense']).groupby('Date')['INR'].sum().reset_index()

dfdate['Day'] = ''
dfdate['Month'] = ''
dfdate['Year'] = ''

for i, d in enumerate(dfdate['Date']):
    day = ''
    month = ''
    year = ''

    bar_count = 0
    for caractere in dfdate['Date'][i]:
        if caractere != '/':
            if bar_count == 0:
                day += caractere
            elif bar_count == 1:
                month += caractere
            else:
                year += caractere
        else:
            bar_count += 1
    
    dfdate.loc[[i], 'Day'] = day
    dfdate.loc[[i], 'Month'] = month
    dfdate.loc[[i], 'Year'] = year

dfdate = dfdate.sort_values(['Year', 'Month', 'Day']).reset_index(drop='index')

display(dfdate)

chart = px.histogram(dfdate, x='Date', y='INR', text_auto=True)
chart.show()

,Date,INR,Day,Month,Year
0,21/11/2021,3005.00,21,11,2021
1,22/11/2021,187.00,22,11,2021
2,23/11/2021,229.00,23,11,2021
3,24/11/2021,163.00,24,11,2021
4,25/11/2021,298.00,25,11,2021
...,...,...,...,...,...
82,26/02/2022,3449.00,26,02,2022
83,27/02/2022,243.00,27,02,2022
84,28/02/2022,539.15,28,02,2022
85,01/03/2022,775.00,01,03,2022


In [235]:
# Expenses x Incomes

chart = px.pie(df, names='Income/Expense', title='Expenses x Incomes', values='INR')
chart.show()

In [236]:
# Category that people much expense

# for c in df['Category'].unique():
#     print(f"'{c}'", end=', ')

# categories = ('Food', 'Other', 'Transportation', 'Social Life', 'Household', 'Apparel', 'Education', 'Salary', 'Allowance', 'Self-development', 'Beauty', 'Gift', 'Petty cash')

dfcategory_expense = df.drop(df[df['Income/Expense'] == 'Income'].index).reset_index(drop='index')
dfcategory_expense = dfcategory_expense[['Category', 'INR']]
dfcategory_expense = dfcategory_expense.groupby('Category').sum()
dfcategory_expense = dfcategory_expense.sort_values('INR', ascending=False)

print(f'The most invested category is {dfcategory_expense.iloc[0].name} and the expense is {dfcategory_expense.iat[0, 0]}')

The most invested category is Food and the expense is 24502.48


In [237]:
# Invested categories

display(dfcategory_expense)

,INR
Category,
Food,24502.48
Household,12188.00
Transportation,9203.80
Other,5117.00
Apparel,3388.00
Social Life,2513.72
Education,1400.00
Self-development,400.00
Beauty,196.00
